In [ ]:
# 4　インプット
import random

# 親クラスの作成：プレイヤーとモンスターの共通部分（名前、HP、攻撃力など）を設定する
class Character:
    def __init__(self, name, HP, attack_power): # インスタンス化で使う属性の設定をする
        self.name   = name         # 名前
        self.hp     = HP           # HP
        self.attack = attack_power # 攻撃力

    # ①プレイヤー・モンスターの共通の攻撃処理は親クラスに書く（誰を攻撃するかは引数で渡す）
    def common_attack(self, target):      # ※target_monster、playerと違う引数なのに処理できるの第2引数同士だから
        attack_dice = random.randint(1,6) # 1-6が出るサイコロを用意

        if attack_dice == 1:              #　サイコロが1なら攻撃力2倍
            damage = self.attack * 2      # ※プレイヤーの番ならself＝プレイヤー。プレイヤーの攻撃として処理している
            damage_text = f"会心の一撃！{target.name}に{damage}のダメージ。"  # ※プレイヤーの番なら自分のことではないからtargetをつける

        elif attack_dice == 2:            # サイコロが2なら攻撃ミス
            damage = 0
            damage_text = f"ミス！{target.name}は攻撃を受けない。"

        else:                             # それ以外は規定の攻撃
            damage = self.attack
            damage_text = f"{target.name}に{damage}のダメージ。"

        target.hp -= damage  # プレイヤーの残りのHPの計算
        return damage_text   # ※返すのはdamage_textだけ。target.hpは返さなくても更新され、今のターゲットと分かるようになっている


# ②モンスタークラスの作成（Characterを継承しているので、Character のメソッドも使える）
class Monster(Character):
    def do_attack(self, player):   # モンスターの攻撃を定義
        common_attack_text = self.common_attack(player) # 上記①の共通攻撃のreturnを引用。　※result = 「{target.name}に{damage}のダメージ」など。出力はしない
        if player.hp <= 0 :        # プレイヤーのHPが0より少なくなったら、マイナスは0に修正し、気絶したと表示
            player.hp = 0
            return f"{self.name}の攻撃。{common_attack_text}{player.name}のHPは{player.hp}。{target_player.name}は気絶した＞＜。\n"
        else:
            return f"{self.name}の攻撃。{common_attack_text}{player.name}のHPは{player.hp}。"


# ③プレイヤークラス（Characterを継承しているので、Character のメソッドも使える）
class Player(Character):   # プレイヤーの攻撃を定義
    def __init__(self, name, HP, attack_power, MP, magic_power): # インスタンス化で使う属性の設定をする
        super().__init__(name, HP, attack_power)  # 親クラス(Character)を呼ぶ
        self.mp = MP                              # 親クラスにMPを追加
        self.magic = magic_power                  # 親クラスに魔法力を追加
        self.max_hp = HP                          # 最大HPの設定（インスタン化の時のHPと同じ値なので、引数として渡さなくていいので、↑には書かない）

    # プレイヤーの攻撃を定義(騎士と、魔法使いの一部)
    def do_attack(self, target_monster):
        print(f"{self.name}の攻撃。")

        common_attack_text = self.common_attack(target_monster) # ①上記①の共通攻撃のreturnを引用。「{target.name}に{damage}のダメージ」など。出力はしない
        if target_monster.hp <= 0 :               # ターゲットモンスターのHPが0より少なくなったら、マイナスは0に修正し、ターゲットモンスターを倒したことを表示
            target_monster.hp = 0
            return f"\n{common_attack_text}{target_monster.name}のHPは{target_monster.hp}。{target_monster.name}を倒した。\n" # 「キメラAに15のダメージ。 キメラAのHPは13。」
        else:
            return f"\n{common_attack_text}{target_monster.name}のHPは{target_monster.hp}。\n"

    # 魔法使いの攻撃を定義
    def magic_attack(self, target_monster):
        print(f"{self.name}の攻撃。")

        atack_kinds_dice = int(input("1：攻撃、2：魔法攻撃、3：回復。　選択→"))  # 物理攻撃、魔法攻撃、回復を選択する

        # 物理攻撃
        if atack_kinds_dice == 1:
            return self.do_attack(target_monster) # 物理攻撃は上と同じ処理を行う

        # 魔法攻撃
        elif atack_kinds_dice == 2:
                magic_dice = int(input("1：メラ、2：イオ。　選択→")) # さらに魔法の種類を選択

                #2-1 メラ
                if magic_dice == 1:
                    if self.mp < 2:             # まずMP確認。MPが2より少なかったら、足りないことを表示して、攻撃せずに終わる
                        return(f"{self.name}はメラを唱えた。MPが足りない！(現在MP: {self.mp})")
                    else:
                        self.mp -= 2            # メラ：MP2減らす
                        damage = self.magic * 2 # メラ:ダメージ10
                        magic_damage_text = f"{self.name}はメラを唱えた。{target_monster.name}に{damage}のダメージ"

                    target_monster.hp -= damage # モンスターの残りのHPの計算

                    if target_monster.hp <= 0:  # ターゲットのモンスターのHPが0より少なくなったら、マイナスは0に修正し、ターゲットモンスターを倒したことを表示
                        target_monster.hp = 0
                        return(f"\n{magic_damage_text}。{target_monster.name}のHPは{target_monster.hp}。{target_monster.name}を倒した\n")
                    else:
                        return(f"\n{magic_damage_text}。{target_monster.name}のHPは{target_monster.hp}。\n")

                #2-2 イオ
                elif magic_dice == 2:
                    global monsters # 下の↓monstersリストを呼ぶ

                    if self.mp < 10:          # まずMP確認。MPが10より少なかったら、足りないことを表示して、攻撃せずに終わる
                        return(f"{self.name}はイオを唱えた。MPが足りない！(現在MP: {self.mp})")
                    else:
                        self.mp -= 10            # イオ：MP10減らす
                        damage = self.magic * 4  # イオ：ダメージ20
                        io_text = f"{self.name}はイオを唱えた。モンスター全員に{damage}のダメージ。\n"

                        # （残っているのモンスターを加える用）新しいモンスターリストを用意
                        new_monsters_list = []

                        for monster in monsters:
                            monster.hp -= damage    # 各モンスターの残りのHPの計算

                            if monster.hp <= 0:     # ターゲットのモンスターのHPが0より少なくなったら、マイナスは0に修正し、ターゲットモンスターを倒したことを表示
                                monster.hp = 0

                                io_text += f"{monster.name}のHPは{monster.hp}。{monster.name}を倒した。\n"
                            else:
                                io_text += f"{monster.name}のHPは{monster.hp}。\n"
                                new_monsters_list.append(monster)  # 生き残っている場合は新しいリストに加える

                        # リストを更新
                        monsters = new_monsters_list
                        return io_text    # モンスタ一1人1人の処理コメント（HPの変化や倒れたなど）をまとめて返す

                        if not monsters:  # リストに、モンスターがいないときは「敵をすべて倒した」と表示
                           io_text += "！！敵をすべて倒した！！\n"

        # 回復魔法
        else:
            if self.mp < 3:               # まずMP確認。MPが3より少なかったら、足りないことを表示して、何もせずに終わる
                return(f"{self.name}はホイミを唱えた。MPが足りない！(現在MP: {self.mp})")
            else:
                self.mp -= 3              # ホイミ：MP3減らす
                recovery = self.magic * 6 # ホイミ：回復30

                # 回復対象の選択
                target_player = int(input("1：騎士、2：魔法使い。　選択→"))
                if target_player == 1:
                    target_player = knight
                elif target_player == 2:
                    target_player = wizard

                target_player.hp += recovery                # 回復後のHPを求める
                if target_player.hp > target_player.max_hp: # 最大HPを超えないようにする
                    target_player.hp = target_player.max_hp
                return(f"\n{self.name}はホイミを唱えた。{target_player.name}のHPは{target_player.hp}に回復した。\n")


# クラスを元にインスタンス化。最初の情報を入力
kimeraA  = Monster("キメラA",  30, 8) # モンスター名、最初のHP、攻撃力を入力
kimeraB  = Monster("キメラB",  30, 8)
suraimuA = Monster("スライムA",10, 4)
suraimuB = Monster("スライムB",10, 4)
dorakiA  = Monster("ドラキーA",25, 3)
dorakiB  = Monster("ドラキーB",25, 3)

knight   = Player("騎士",    70,15,  0, 0) # プレイヤー名、最初のHP、攻撃力、最初のMP、魔法力を入力
wizard   = Player("魔法使い",40, 5, 16, 5)


# プレイヤー、モンスターたちのリストを用意
players  = [knight, wizard]
monsters = [kimeraA, kimeraB, suraimuA, suraimuB, dorakiA, dorakiB]

num = 0
while players and monsters:     # リストにプレイヤーもモンスターもいる時は繰り返す（monstersと書くだけで誰かがいることを指す）
    num += 1
    print(f"\n●{num}ターン目●")

    # 騎士の攻撃
    target_monster = random.choice(monsters)      # ターゲットにするモンスターはランダム
    if knight.hp > 0 :
        print(knight.do_attack(target_monster))   # ③の騎士の攻撃を実行。引数にターゲットモンスターを渡す　※→「☆☆プレイヤーの攻撃☆☆ キメラAに15のダメージ。 キメラAのHPは13。」
                                                  # ※第一引数のselfは隠れている。なのでdef do_attack(self, target_monster): のtarget_monsterとは同じ第2引数と認識される
    #モンスター生存判定
    new_monsters_list = []
    for monster in monsters:
        if monster.hp > 0 :                    # モンスターのHPが0より大きい時は、モンスターを新しいリストに加える
            new_monsters_list.append(monster)

    # 残っているモンスターだけでリストを更新
    monsters = new_monsters_list
    if not monsters:                           # モンスターがリストにいなかったら終了
        print(f"！！敵をすべて倒した！！")
        break


    # 魔法使いの攻撃
    target_monster = random.choice(monsters)       # ターゲットにするモンスターはランダム
    if wizard.hp > 0 :
        print(wizard.magic_attack(target_monster)) # ③魔法使いの攻撃を実行

    #モンスター生存判定
    new_monsters_list = []
    for monster in monsters:
        if monster.hp > 0 :                    # モンスターのHPが0より大きい時は、そのモンスターを加える)
            new_monsters_list.append(monster)  # new_list に加える

    # 残っているモンスターだけでリストを更新
    monsters = new_monsters_list
    if not monsters:                           # 全てのモンスターがリストからいなくなったら終了
        print(f"！！敵をすべて倒した！！")
        break



    # モンスターの攻撃
    random.shuffle(monsters)                     # モンスターが攻撃する順番をランダムにする

    for monster in monsters:
        target_player = random.choice(players)   # ターゲットにするプレイヤーはランダム
        print(monster.do_attack(target_player))  # ②モンスターの攻撃を実行。引数にターゲットを渡す

        new_player_list = []
        for player in players:                  # 各プレイヤーの生存判定
            if player.hp > 0 :                  # HPが0より大きいプレイヤーは新しいリストに加える
                new_player_list.append(player)

        # 残っているプレイヤーだけでリストを更新
        players = new_player_list

        if not players:                         # 全てのプレイヤーがリストからいなくなったらゲームオーバー
            print(f"～ゲームオーバー～")
            break
